# Proyecto 2
# Procesamiento de Lenguaje Natural basados en arquitecturas neuronales secuenciales.

## Librerias

In [1]:
import os
import re
import pandas as pd
import spacy
import random

## Preparación del Corpus

In [2]:
# Descargar el modelo de spaCy para español si no está ya descargado
# python -m spacy download es_core_news_sm

# Cargar el modelo de spaCy para español
nlp = spacy.load("es_core_news_sm")

# Rutas
RAW_PATH = "Data/raw_corpus.txt"          # entrada
OUT_DIR = "Data/processed"                # carpeta de salida

os.makedirs(OUT_DIR, exist_ok=True)

# Cargar el texto bruto
with open(RAW_PATH, "r", encoding="utf-8") as f:
    raw_text = f.read()

print("Longitud del texto bruto:", len(raw_text))
print(raw_text[:500])  # muestra un fragmento

Longitud del texto bruto: 3975
Érase un pobre campesino que estaba una noche junto al hogar atizando el fuego, mientras su mujer hilaba, sentada a su lado.

Dijo el hombre: – ¡Qué triste es no tener hijos! ¡Qué silencio en esta casa, mientras en las otras todo es ruido y alegría! – Sí -respondió la mujer, suspirando-. Aunque fuese uno solo, y aunque fuese pequeño como el pulgar, me daría por satisfecha. Lo querríamos más que nuestra vida.

Sucedió que la mujer se sintió descompuesta, y al cabo de siete meses trajo al mundo un


### Limpieza básica

In [3]:
def clean_text(text: str) -> str:
    # Normalizar saltos de línea: quitar espacios antes de saltos
    text = re.sub(r"[ \t]+\n", "\n", text)

    # Normalizar comillas curvas a comillas rectas (opcional)
    text = text.replace("“", "\"").replace("”", "\"").replace("’", "'").replace("‘", "'")
    
    # Unificar guiones largos de diálogo (si aparecen variantes)
    text = text.replace("—", "–")
    
    # Quitar espacios repetidos
    text = re.sub(r"[ \t]+", " ", text)
    
    # Quitar saltos de línea múltiples excesivos (más de 2)
    text = re.sub(r"\n{3,}", "\n\n", text)
    
    # Strip global
    text = text.strip()
    return text

clean_corpus = clean_text(raw_text)

print("Longitud del texto limpio:", len(clean_corpus))
print(clean_corpus[:500])

Longitud del texto limpio: 3975
Érase un pobre campesino que estaba una noche junto al hogar atizando el fuego, mientras su mujer hilaba, sentada a su lado.

Dijo el hombre: – ¡Qué triste es no tener hijos! ¡Qué silencio en esta casa, mientras en las otras todo es ruido y alegría! – Sí -respondió la mujer, suspirando-. Aunque fuese uno solo, y aunque fuese pequeño como el pulgar, me daría por satisfecha. Lo querríamos más que nuestra vida.

Sucedió que la mujer se sintió descompuesta, y al cabo de siete meses trajo al mundo un


In [4]:
# Guardar el texto limpio
clean_corpus_path = os.path.join(OUT_DIR, "clean_corpus.txt").replace("\\", "/")
with open(clean_corpus_path, "w", encoding="utf-8") as f:
    f.write(clean_corpus)

print(f"Texto limpio guardado en: {clean_corpus_path}")

Texto limpio guardado en: Data/processed/clean_corpus.txt


### Tokenizacion y Segmentacion en oraciones

In [5]:
# Procesar todo el corpus con spaCy
doc = nlp(clean_corpus)

sentences = []
tokens_per_sentence = []

for sent in doc.sents:
    sent_text = sent.text.strip()
    if not sent_text:
        continue
    
    # Lista de tokens de la oración, sin espacios
    toks = [tok.text for tok in sent if not tok.is_space]
    
    # Filtramos oraciones muy cortas si quieres (opcional)
    if len(toks) == 0:
        continue
    
    sentences.append(sent_text)
    tokens_per_sentence.append(toks)

print("Número de oraciones detectadas:", len(sentences))
for i in range(min(5, len(sentences))):
    print(f"\nOración {i}: {sentences[i]}")
    print("Tokens:", tokens_per_sentence[i])


Número de oraciones detectadas: 41

Oración 0: Érase un pobre campesino que estaba una noche junto al hogar atizando el fuego, mientras su mujer hilaba, sentada a su lado.
Tokens: ['Érase', 'un', 'pobre', 'campesino', 'que', 'estaba', 'una', 'noche', 'junto', 'al', 'hogar', 'atizando', 'el', 'fuego', ',', 'mientras', 'su', 'mujer', 'hilaba', ',', 'sentada', 'a', 'su', 'lado', '.']

Oración 1: Dijo el hombre: – ¡Qué triste es no tener hijos! ¡Qué silencio en esta casa, mientras en las otras todo es ruido y alegría! –
Tokens: ['Dijo', 'el', 'hombre', ':', '–', '¡', 'Qué', 'triste', 'es', 'no', 'tener', 'hijos', '!', '¡', 'Qué', 'silencio', 'en', 'esta', 'casa', ',', 'mientras', 'en', 'las', 'otras', 'todo', 'es', 'ruido', 'y', 'alegría', '!', '–']

Oración 2: Sí -respondió la mujer, suspirando-.
Tokens: ['Sí', '-respondió', 'la', 'mujer', ',', 'suspirando-', '.']

Oración 3: Aunque fuese uno solo, y aunque fuese pequeño como el pulgar, me daría por satisfecha.
Tokens: ['Aunque', 'fuese',

In [6]:
tokenized_sentences_str = [" ".join(toks) for toks in tokens_per_sentence]

data_df = pd.DataFrame({
    "id": list(range(len(sentences))),
    "sentence": sentences,
    "tokens": tokenized_sentences_str
})

all_sentences_path = os.path.join(OUT_DIR, "sentences_tokenized_spacy.csv").replace("\\", "/")
data_df.to_csv(all_sentences_path, index=False, encoding="utf-8")
print(f"Datos tokenizados guardados en: {all_sentences_path}")


Datos tokenizados guardados en: Data/processed/sentences_tokenized_spacy.csv


### División de entrenamiento, validación y prueba

In [7]:
indices = list(range(len(data_df)))
random.shuffle(indices)

n_total = len(indices)
n_train = int(0.8 * n_total)
n_val = int(0.1 * n_total)
n_test = n_total - n_train - n_val

train_idx = indices[:n_train]
val_idx = indices[n_train:n_train + n_val]
test_idx = indices[n_train + n_val:]

train_df = data_df.iloc[train_idx].reset_index(drop=True)
val_df = data_df.iloc[val_idx].reset_index(drop=True)
test_df = data_df.iloc[test_idx].reset_index(drop=True)

print("Tamaños:")
print("train:", len(train_df))
print("val:  ", len(val_df))
print("test: ", len(test_df))

train_path = os.path.join(OUT_DIR, "train_sentences_spacy.csv").replace("\\", "/")
val_path = os.path.join(OUT_DIR, "val_sentences_spacy.csv").replace("\\", "/")
test_path = os.path.join(OUT_DIR, "test_sentences_spacy.csv").replace("\\", "/")

train_df.to_csv(train_path, index=False, encoding="utf-8")
val_df.to_csv(val_path, index=False, encoding="utf-8")
test_df.to_csv(test_path, index=False, encoding="utf-8")

print("\nConjuntos guardados en:")
print("- Train:", train_path)
print("- Validation:", val_path)
print("- Test:", test_path)

Tamaños:
train: 32
val:   4
test:  5

Conjuntos guardados en:
- Train: Data/processed/train_sentences_spacy.csv
- Validation: Data/processed/val_sentences_spacy.csv
- Test: Data/processed/test_sentences_spacy.csv
